### Implementation of Neural Network


<p align="center">
<img src="images/day63_01.png" width="400">
        $$\mathbf{h} = \text{sigmoid}(\mathbf{xW_1}+\mathbf{b_1})$$
    $$\mathbf{\hat{y}} = \text{softmax}(\mathbf{hW_2}+\mathbf{b_2})$$

<br>
Figure 1. Image from CS224n NLP with Deep Learning
</p>


In [1]:
import numpy as np
import random

In [2]:
def softmax(x):
    #matrix
    if len(x.shape) > 1:
        c = -1*x.max(axis=1, keepdims=True)
        x = x + c
        x_exp = np.exp(x)
        x = x_exp/np.sum(x_exp, axis=1, keepdims=True)
    #vector
    else:
        c = -1*x.max()
        x = x + c
        x_exp = np.exp(x)
        x = x_exp/np.sum(x_exp)
    return x

In [3]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def grad_sigmoid(s):
    # s = sigmoid(x)
    return s*(1-s)


In [4]:
def forward_backward_pass(X, labels, params, dimensions):
    ofs = 0
    Dx, H, Dy = (dimensions[0], dimensions[1], dimensions[2])
    
    W1 = np.reshape(params[ofs:ofs + Dx*H], (Dx,H))
    ofs += Dx * H
    b1 = np.reshape(params[ofs:ofs + H], (1,H))
    ofs += H
    W2 = np.reshape(params[ofs:ofs + H*Dy], (H,Dy))
    ofs += H*Dy
    b2 = np.reshape(params[ofs:ofs + Dy], (1,Dy))
    
    # Forward pass
    z1 = np.dot(X,W1) + b1
    h = sigmoid(z1)
    z2 = np.dot(h,W2) + b2
    y = softmax(z2)
    
    cost = -1*np.sum(labels*np.log(y))
    
    #Backward pass
    gradz2 = y - labels
    gradW2 = np.dot(h.T, gradz2)
    gradb2 = gradz2
    gradW1 = np.dot(X.T, grad_sigmoid(h) * np.dot(gradz2, W2.T))
    gradb1 = np.sum(grad_sigmoid(h) * np.dot(gradz2, W2.T), axis=0, keepdims=True)
    
    grad = np.concatenate((gradW1.flatten(), gradb1.flatten(), 
                           gradW2.flatten(), gradb2.flatten()))
    
    return cost, grad, y

In [5]:
def gradient_checking(f, x):
    fx, grad, _ = f(x)
    h = 1e-4
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    while not it.finished:
        ix = it.multi_index        
        xtemp = x[ix]
        
        x[ix] = x[ix] + h
        f_plus, _, _ = f(x)
        x[ix] = xtemp
        
        x[ix] = x[ix] - h
        f_minus, _, _ = f(x)
        x[ix] = x_temp
        
        numgrad = (f_plus - f_minus)/(2*h)
        
        reldiff = abs(numgrad - grad[ix]) / max(1, abs(numgrad), abs(grad[ix]))
        if reldiff > 1e-5:
            print("Gradient check failed.")
            print("First gradient error found at index %s" % str(ix))
            print("Your gradient: %f \t Numerical gradient: %f" % (
                grad[ix], numgrad))
            return

        it.iternext() # Step to next dimension
    print("Gradient check passed!")

In [6]:
def update_parameters(params, grad, learning_rate):
    for ix in range(0, len(params)):
        params[ix] = params[ix] - learning_rate * grad[ix]
        
    return params

In [7]:
def accuracy(labels, y):
    m, _ = labels.shape
    idx_labels = np.argmax(labels, axis=1)
    idx_y = np.argmax(y, axis=1)    
    count = np.sum(idx_labels == idx_y)
    
    return count / m

In [8]:
N = 20
dimensions = [10, 5, 10]
data = np.random.randn(N, dimensions[0])   # each row will be a datum
labels = np.zeros((N, dimensions[2]))
for i in range(N):
    labels[i, random.randint(0,dimensions[2]-1)] = 1

params = np.random.randn((dimensions[0] + 1) * dimensions[1] + (
        dimensions[1] + 1) * dimensions[2], )

n_iterations = 10000
learning_rate = 0.01

for i in range(0, n_iterations):
    cost, grad, y_pred = forward_backward_pass(data, labels, params, dimensions)
    params = update_parameters(params, grad, learning_rate)
    
    if i % 500 == 0:
        print("Iteration %d \t cost: %f \t accuracy: %f"%(i, cost, accuracy(labels, y_pred)))
print("-"*10)
print("final cost %f"%(cost))
print("accuracy %f"%(accuracy(labels, y_pred)))

Iteration 0 	 cost: 59.859510 	 accuracy: 0.100000
Iteration 500 	 cost: 15.568468 	 accuracy: 0.750000
Iteration 1000 	 cost: 9.455554 	 accuracy: 0.850000
Iteration 1500 	 cost: 5.064316 	 accuracy: 0.950000
Iteration 2000 	 cost: 3.248469 	 accuracy: 0.950000
Iteration 2500 	 cost: 2.361378 	 accuracy: 1.000000
Iteration 3000 	 cost: 1.596314 	 accuracy: 1.000000
Iteration 3500 	 cost: 1.169276 	 accuracy: 1.000000
Iteration 4000 	 cost: 0.925379 	 accuracy: 1.000000
Iteration 4500 	 cost: 0.768001 	 accuracy: 1.000000
Iteration 5000 	 cost: 0.657157 	 accuracy: 1.000000
Iteration 5500 	 cost: 0.574506 	 accuracy: 1.000000
Iteration 6000 	 cost: 0.510382 	 accuracy: 1.000000
Iteration 6500 	 cost: 0.459141 	 accuracy: 1.000000
Iteration 7000 	 cost: 0.417242 	 accuracy: 1.000000
Iteration 7500 	 cost: 0.382340 	 accuracy: 1.000000
Iteration 8000 	 cost: 0.352819 	 accuracy: 1.000000
Iteration 8500 	 cost: 0.327524 	 accuracy: 1.000000
Iteration 9000 	 cost: 0.305610 	 accuracy: 1.00